In [1]:
#>> Contains layout and callbacks for app 1
''' 
Notes:
This file is for creating app 1, it ontains layout and callbacks for app 1
'''
# Import packages---------------------------------------------------------------------------------------------
## Data Acess and Manipulation
import numpy as np

## Import App Packages
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import dash_bootstrap_components as dbc
from datetime import datetime as dt
from dash.exceptions import PreventUpdate
import pandas as pd
import io
from flask import send_file

## Data Visualization
import plotly.graph_objs as go

##Local Imports
from src.utils.data_loaders import get_excel, get_data_from_sf, get_geojson, load_dict, get_geopandas
from src.utils.queries import data_query_order_line, data_query_item_weights
from src.utils.data_processing import data_preprocessing, data_preprocessing_routes, calculate_metrics, clean_routes, data_preprocessing_order_details, data_preprocessing_choropleth
from src.utils.map import maps, route_maps
from src.utils.vehicle_route_planning import DistanceCalculator, create_data_model, run_model, get_num_trucks
from src.utils.routes_processing import enhance_optimized_route

from app import app

## Utilities
import os
from dotenv import load_dotenv 
load_dotenv()

df_order_line_1 = get_data_from_sf(data_query_order_line)

import os
from dotenv import load_dotenv 
load_dotenv()
df_weights = pd.read_excel(os.getenv("item_weight"), usecols=['itemCode', 'itemWeight'])
df_weights.head()

In [2]:

# Incorporate data---------------------------------------------------------------------------------------------
# Loading Data
#df_order_line = df_order_line_1[1556:1557]# df_order_line_1.head(1700)
df_order_line = df_order_line_1[df_order_line_1.index !=1556]
#df_weights = get_data_from_sf(data_query_item_weights)
df_weights = pd.read_excel(os.getenv("item_weight"), usecols=['itemCode', 'itemWeight'])
js = get_geojson(os.getenv("geojson_loc_2"))
geoJSON = get_geojson(os.getenv("geojson_loc_3"))
geo_df = get_geopandas(os.getenv("geojson_loc_3"))
bbox = (22.4969475367, 26.055464179, 51.5795186705, 56.3968473651)  # Set only if using 'GRAPH' method
Map2_loc = os.getenv("Map2_location")

# Preprocessing the Data
df = data_preprocessing(df_order_line, df_weights, js) 
order_details = data_preprocessing_order_details(df, ['All'])

# Distance Matrics
distance_calculator = DistanceCalculator(bbox=bbox, network_type="drive") 
id, coordinates, demand, dm = distance_calculator.df_to_dm(order_details, 'customerLat', 'customerLong', 'customerCode', 'totalWeight', 'base', 'GD')
#data = create_data_model(demand, dm)


In [3]:
# Solve & Load the CVRP problem
data = create_data_model(demand, dm)
print(data)


{'distance_matrix': array([[    0,  8295,  8694, ...,  8584,  8742,  4334],
       [ 8295,     0, 16459, ..., 16338, 16503,  5056],
       [ 8694, 16459,     0, ...,   124,    48, 13028],
       ...,
       [ 8584, 16338,   124, ...,     0,   165, 12918],
       [ 8742, 16503,    48, ...,   165,     0, 13075],
       [ 4334,  5056, 13028, ..., 12918, 13075,     0]]), 'demands': [0, 7, 0, 41, 29, 2, 41, 4, 100, 0, 9, 11, 0, 12, 10, 0, 27, 0, 13, 0, 211, 107, 31, 12, 17, 0, 75, 0, 28, 51, 22, 0, 58, 96, 7, 0, 0, 23, 9, 0, 13, 24, 3, 0, 0, 0, 0, 0, 0, 0, 24, 58, 12, 1, 0, 0, 0, 0, 0, 67, 20, 52, 0, 0, 3, 33, 0, 156, 0, 3, 0, 0, 0, 0, 12, 0, 0, 0, 615, 0, 0, 39, 48, 0, 0, 0, 22, 65, 13, 0, 0, 0, 0, 21, 70, 0, 2, 50, 0, 0, 15, 0, 0, 0, 18, 0, 0, 21, 0, 0, 324, 30, 0, 604, 61, 85, 0, 7, 530, 910, 58, 19, 32, 630, 1230, 521, 0, 9, 5, 25, 51, 54, 86, 317, 303, 0, 345, 14, 10, 21, 2, 9, 426, 22, 81, 5, 119, 82, 260, 2, 458, 40, 482, 87, 353, 0, 47, 33, 15, 577, 86, 0, 35, 0, 15, 0, 72, 17, 0, 8

In [4]:
output= run_model(data)
print(output)

(<ortools.constraint_solver.pywrapcp.RoutingIndexManager; proxy of <Swig Object of type 'operations_research::RoutingIndexManager *' at 0x0000015154FA2F40> >, <ortools.constraint_solver.pywrapcp.RoutingModel; proxy of <Swig Object of type 'operations_research::RoutingModel *' at 0x0000015164E1F0F0> >, <ortools.constraint_solver.pywrapcp.Assignment; proxy of <Swig Object of type 'operations_research::Assignment *' at 0x00000151448873F0> >, {0: {'route_plan': [0], 'cumulative_route_load': [0]}, 1: {'route_plan': [0], 'cumulative_route_load': [0]}, 2: {'route_plan': [0], 'cumulative_route_load': [0]}, 3: {'route_plan': [0], 'cumulative_route_load': [0]}, 4: {'route_plan': [0], 'cumulative_route_load': [0]}, 5: {'route_plan': [0], 'cumulative_route_load': [0]}, 6: {'route_plan': [0], 'cumulative_route_load': [0]}, 7: {'route_plan': [0], 'cumulative_route_load': [0]}, 8: {'route_plan': [0], 'cumulative_route_load': [0]}, 9: {'route_plan': [0], 'cumulative_route_load': [0]}, 10: {'route_plan

In [5]:
output = load_dict(os.getenv("model_output_location"))
# Find the routes
route_output = enhance_optimized_route(output, coordinates, id)

API call failed for this list of coordinates coordinates:  [[55.06185223067843, 24.931690466392208], ['55.1762197', '24.9789814'], ['55.18360903', '24.98594083'], ['55.8023118', '25.2196159'], ['55.85933605', '25.28101319'], ['55.87840707', '25.28138983'], ['55.87972065', '25.28470953'], ['56.33393486', '25.16244746'], ['56.33692932', '25.16572974'], ['56.33979114', '25.16425763'], ['56.33976194', '25.16657223'], ['56.34810491', '25.23825772'], ['56.3601253', '25.271208'], ['56.36021631', '25.27326995'], ['56.36214435', '25.27359185'], ['56.36253168', '25.28865974'], ['56.36497572', '25.29275516'], ['56.3640989', '25.2927652'], ['56.36264719', '25.29495949'], ['56.34961639', '25.34440223'], ['56.35427397', '25.34548413'], ['56.35522958', '25.51571477'], ['56.2616838', '25.5797136'], ['55.85335866', '25.35263409'], ['55.85301838', '25.35209073'], ['55.85257973', '25.34117057'], ['55.4635358', '25.1573008'], ['55.380675000000004', '25.10867166666667'], ['55.24572098', '25.04049633']]
{"e

In [19]:
# Preprocessing the Data
route_output_df = clean_routes(route_output) 
route_output_df.tail(20)

,route_geometry,route_distance,route_duration,route_plan,cumulative_route_load,route_coords,route_ids,total_route_load,total_customers,Vehicle ID,ID
20,None,1.0,1.0,"[0, 317, 435, 253, 217, 180, 423, 324, 325, 32...","[0, 21, 54, 55, 55, 328, 332, 341, 350, 365, 5...","[[55.06185223067843, 24.931690466392208], [55....","[0, 20223088, 6987, 20221766, 20221290, 19228,...",2486,28,Vehicle 20,20
21,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,474.6,432.2,"[0, 70, 84, 366, 364, 237, 227, 485, 486, 86, ...","[0, 0, 0, 0, 0, 0, 12, 12, 12, 34, 315, 315, 3...","[[55.06185223067843, 24.931690466392208], [55....","[0, 128, 135, 432, 416, 20221519, 20221450, 84...",2994,29,Vehicle 21,21
22,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,404.4,353.9,"[0, 498, 160, 309, 367, 444, 233, 482, 132, 37...","[0, 41, 127, 128, 169, 194, 244, 245, 331, 357...","[[55.06185223067843, 24.931690466392208], [55....","[0, 8774, 17927, 20222865, 4394, 7258, 2022150...",2732,29,Vehicle 22,22
23,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,313.1,284.3,"[0, 507, 510, 116, 504, 187, 186, 479, 389, 42...","[0, 14, 14, 14, 96, 114, 125, 134, 161, 389, 4...","[[55.06185223067843, 24.931690466392208], [54....","[0, 9075, 9243, 16328, 8942, 19554, 19487, 812...",1073,27,Vehicle 23,23
24,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,290.8,245.9,"[0, 358, 432, 238, 55, 54, 53, 52, 130, 246, 3...","[0, 0, 1, 28, 28, 28, 29, 41, 92, 99, 129, 129...","[[55.06185223067843, 24.931690466392208], [55....","[0, 368, 6945, 20221520, 12180, 12174, 12173, ...",2981,29,Vehicle 24,24
25,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,285.1,259.8,"[0, 297, 296, 74, 94, 102, 3, 28, 26, 98, 29, ...","[0, 0, 0, 12, 82, 82, 123, 151, 226, 226, 277,...","[[55.06185223067843, 24.931690466392208], [55....","[0, 20222656, 20222655, 13067, 14021, 14886, 1...",1103,29,Vehicle 25,25
26,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,145.0,132.7,"[0, 97, 12, 9, 122, 131, 34, 213, 5, 221, 36, ...","[0, 50, 50, 50, 82, 136, 143, 159, 161, 252, 2...","[[55.06185223067843, 24.931690466392208], [54....","[0, 14563, 10195, 10162, 16460, 16782, 11105, ...",726,27,Vehicle 26,26
27,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,135.7,121.3,"[0, 156, 117, 240, 251, 151, 523, 212, 526, 51...","[0, 47, 54, 54, 94, 134, 160, 299, 353, 353, 3...","[[55.06185223067843, 24.931690466392208], [54....","[0, 17796, 16361, 20221525, 20221665, 17653, 9...",1483,28,Vehicle 27,27
28,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,170.2,149.6,"[0, 403, 211, 332, 500, 374, 375, 445, 378, 19...","[0, 39, 39, 66, 82, 98, 139, 139, 233, 233, 38...","[[55.06185223067843, 24.931690466392208], [54....","[0, 5411, 20221, 20409, 8865, 4754, 4805, 7318...",1408,29,Vehicle 28,28
29,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,491.8,356.0,"[0, 522, 13, 178, 225, 278, 289, 231, 301, 293...","[0, 15, 27, 58, 64, 65, 65, 71, 71, 71, 71, 71...","[[55.06185223067843, 24.931690466392208], [55....","[0, 9711, 10291, 19096, 20221433, 20222451, 20...",2334,29,Vehicle 29,29


In [15]:
def data_preprocessing_routes_1(order_details, route_output_df):
    
    order_details_clean = order_details[order_details['base']==0].copy()
    order_details_clean.columns = ['Customer Code', 'Emirate Name', 'Customer Name', 'Customer Address', 'Customer Region',
    'Customer Lat', 'Customer Long', 'Total Orders', 'Total Weight',
    'Total Items', 'Invoice Quantity', 'Sales Value', 'Cost Value',
    'Gross Profit', 'Gross Profit Margin', 'Base']

    # Explode the customer_IDs list into separate rows
    exploded_df = route_output_df[['Vehicle ID', 'ID', 'route_ids']].explode('route_ids')
    # Calculate the sequence number of each customer ID per vehicle
    exploded_df['Customer Sequence'] = exploded_df.groupby('Vehicle ID').cumcount() + 1
    # Merge exploded_df with customer_master_df
    result_df = pd.merge(order_details_clean, exploded_df, right_on='route_ids', left_on ='Customer Code', how='left')

    route_output_grouped = result_df.groupby(['Vehicle ID', 'ID']).agg({
                                    'Customer Code': lambda x: pd.Series.nunique(x),
                                    'Customer Region': lambda x: x.unique(),
                                    'Total Orders': 'sum',
                                    'Total Weight': 'sum',
                                    'Sales Value': 'sum',
                                    'Cost Value': 'sum',
                                    'Gross Profit': 'sum'
                                    }
                                ).sort_values(by='ID').reset_index().copy()
    route_output_grouped_df = pd.merge(route_output_grouped, 
                                       route_output_df[['Vehicle ID', 'route_distance', 'route_duration', 'route_plan', 
                                                        'route_coords', 'route_ids', 'total_route_load', 'total_customers']],
                                        right_on='Vehicle ID', left_on ='Vehicle ID', how='left')
    route_output_grouped_df.columns = ['Vehicle ID', 'ID', 'No of Customers', 'Route Regions', 'Total Route Orders',
                                    'Total Route Weight', 'Sales Value', 'Cost Value', 'Gross Profit',
                                    'Total Route Distance', 'Total Route Duration', 'Route Plan', 'Route Coords',
                                    'Route IDs', 'Total Route Load', 'Total Customers']
    
    route_output_grouped_df['Total Route Weight'] = route_output_grouped_df['Total Route Weight'].astype(int)
    route_output_grouped_df['Sales Value'] = route_output_grouped_df['Sales Value'].astype(int)
    route_output_grouped_df['Cost Value'] = route_output_grouped_df['Cost Value'].astype(int)
    route_output_grouped_df['Gross Profit'] = route_output_grouped_df['Gross Profit'].astype(int)
    route_output_grouped_df['Total Route Distance'] = route_output_grouped_df['Total Route Distance'].astype(int)
    route_output_grouped_df['Total Route Duration'] = (route_output_grouped_df['Total Route Duration']/60).astype(int)
    

    return result_df, route_output_grouped_df

In [16]:
# Preprocessing the Data
route_output_df = clean_routes(route_output) # Preprocessing the Data
order_details_clean, route_output_grouped_df = data_preprocessing_routes_1(order_details, route_output_df)



In [17]:
order_details_clean.to_excel('assets/Outputs/order_details_clean_3.xlsx', index=False) 

route_output_grouped_df.to_excel('assets/Outputs/route_output_grouped_df_3.xlsx', index=False) 

In [18]:
route_output_grouped_df.head(2)

,Vehicle ID,ID,No of Customers,Route Regions,Total Route Orders,Total Route Weight,Sales Value,Cost Value,Gross Profit,Total Route Distance,Total Route Duration,Route Plan,Route Coords,Route IDs,Total Route Load,Total Customers
0,Vehicle 20,20,28,"[FUJAIRAH, SHARJAH, AL AIN, UMM AL QUWAIN, DUBAI]",30.0,2495,14604,14444,160,1,0,"[0, 317, 435, 253, 217, 180, 423, 324, 325, 32...","[[55.06185223067843, 24.931690466392208], [55....","[0, 20223088, 6987, 20221766, 20221290, 19228,...",2486,28
1,Vehicle 21,21,29,"[FUJAIRAH, RAS AL KHAIMAH, DUBAI, AJMAN, SHARJ...",31.0,3003,16948,16484,463,474,7,"[0, 70, 84, 366, 364, 237, 227, 485, 486, 86, ...","[[55.06185223067843, 24.931690466392208], [55....","[0, 128, 135, 432, 416, 20221519, 20221450, 84...",2994,29


In [76]:
output['21']
output.items()

dict_items([('0', {'route_plan': [0], 'cumulative_route_load': [0]}), ('1', {'route_plan': [0], 'cumulative_route_load': [0]}), ('2', {'route_plan': [0], 'cumulative_route_load': [0]}), ('3', {'route_plan': [0], 'cumulative_route_load': [0]}), ('4', {'route_plan': [0], 'cumulative_route_load': [0]}), ('5', {'route_plan': [0], 'cumulative_route_load': [0]}), ('6', {'route_plan': [0], 'cumulative_route_load': [0]}), ('7', {'route_plan': [0], 'cumulative_route_load': [0]}), ('8', {'route_plan': [0], 'cumulative_route_load': [0]}), ('9', {'route_plan': [0], 'cumulative_route_load': [0]}), ('10', {'route_plan': [0], 'cumulative_route_load': [0]}), ('11', {'route_plan': [0], 'cumulative_route_load': [0]}), ('12', {'route_plan': [0], 'cumulative_route_load': [0]}), ('13', {'route_plan': [0], 'cumulative_route_load': [0]}), ('14', {'route_plan': [0], 'cumulative_route_load': [0]}), ('15', {'route_plan': [0], 'cumulative_route_load': [0]}), ('16', {'route_plan': [0], 'cumulative_route_load': [0

# API

In [66]:
#coordinates=[[55.06185223067843, 24.931690466392208], ['55.1762197', '24.9789814']]
#coordinates=[[55.06185223067843, 24.931690466392208], ['55.387435', '25.22294'], ['55.390678333333334', '25.229066666666668'], ['55.41598241', '25.26037266'], ['55.42646366', '25.27131671']]
#coordinates=[[55.06185223067843, 24.931690466392208], ['55.387435', '25.22294'], ['55.390678333333334', '25.229066666666668'], ['55.41598241', '25.26037266'], ['55.42646366', '25.27131671'], ['55.44446621', '25.29367289'], ['55.45352128', '25.30965137'], ['55.50274082', '25.38241682'], ['55.50347183', '25.38222953'], ['55.51386421', '25.38809152'], ['55.54621082', '25.43275844'], ['55.7655324', '25.4359959'], ['55.99328359588981', '25.568450647391664'], ['56.00438443', '25.61660573'], ['56.01129975', '25.64259411'], ['55.9660522', '25.7182857'], ['55.94013351', '25.7664422'], ['55.93990468', '25.7889258'], ['55.94184749999999', '25.7911063'], ['55.97137812', '25.79063788'], ['55.9734176', '25.7915691'], ['55.973305', '25.7922914'], ['55.9789504', '25.7930057'], ['55.98876278', '25.80020272'], ['55.9834913', '25.8245779'], ['56.2817241', '25.5996513'], ['56.2727644', '25.5935375'], ['56.2625744', '25.594046'], ['55.73297855', '25.315324'], ['55.4077794961631', '25.1568435737863']]
coordinates=[[55.06185223067843, 24.931690466392208], ['55.387435', '25.22294'], ['55.390678333333334', '25.229066666666668'], ['55.41598241', '25.26037266'], ['55.42646366', '25.27131671'], ['55.44446621', '25.29367289'], ['55.45352128', '25.30965137'], ['55.50274082', '25.38241682'], ['55.50347183', '25.38222953'], ['55.51386421', '25.38809152'], ['55.54621082', '25.43275844'], ['55.7655324', '25.4359959'], ['55.99328359588981', '25.568450647391664'], ['56.00438443', '25.61660573'], ['56.01129975', '25.64259411'], ['55.9660522', '25.7182857'], ['55.94013351', '25.7664422'], ['55.93990468', '25.7889258'], ['55.94184749999999', '25.7911063'], ['55.97137812', '25.79063788'], ['55.9734176', '25.7915691'], ['55.973305', '25.7922914'], ['55.9789504', '25.7930057'], ['55.98876278', '25.80020272'], ['55.9834913', '25.8245779'], ['56.2817241', '25.5996513'], ['56.2727644', '25.5935375'], ['56.2625744', '25.594046'], ['55.73297855', '25.315324'], ['55.4077794961631', '25.1568435737863']]


In [67]:
import requests
import json
import time
body = {"coordinates": coordinates}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624822b79fbe37204f6a83490d4a1566d9a5',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/directions/driving-car', json=body, headers=headers)

In [68]:
call.text

'{"bbox":[55.062485,24.918852,56.282388,25.824472],"routes":[{"summary":{"distance":474597.1,"duration":25934.699999999997},"segments":[{"distance":57191.6,"duration":2474.1,"steps":[{"distance":186.5,"duration":18.2,"type":11,"instruction":"Head northeast","name":"-","way_points":[0,4]},{"distance":327.2,"duration":38.8,"type":7,"instruction":"Enter the roundabout and take the 1st exit","name":"-","exit_number":1,"way_points":[4,7]},{"distance":1172.3,"duration":73.9,"type":0,"instruction":"Turn left onto شارع الخيل الاول, D86","name":"شارع الخيل الاول, D86","way_points":[7,26]},{"distance":1876.5,"duration":98.5,"type":7,"instruction":"Enter the roundabout and take the 1st exit onto شارع مطار آل مكتوم, D53","name":"شارع مطار آل مكتوم, D53","exit_number":1,"way_points":[26,37]},{"distance":1486.9,"duration":86.8,"type":7,"instruction":"Enter the roundabout and take the 2nd exit onto شارع مطار آل مكتوم, D53","name":"شارع مطار آل مكتوم, D53","exit_number":2,"way_points":[37,54]},{"dista

In [69]:


# print(call.status_code, call.reason)
if call.status_code == 200:
    response = json.loads(call.text)
    geometry = response['routes'][0]['geometry']
    distance = round(response['routes'][0]['summary']['distance']/1000,1)
    duration = round(response['routes'][0]['summary']['duration']/60,1)
else:
    print("API call failed for this list of coordinates coordinates: ", coordinates)
    print(call.text)
    geometry = None
    distance = None
    duration = None

In [97]:
print(distance,duration)
len(coordinates)

474.6 432.2


534

In [125]:
def reverse_coordinates(input):
  output = []
  for each in input:
    each.reverse()
    output.append(each)
  return output

def return_coords(coords,ls):
    r_ls = [coords[each] for each in ls]
    r_ls = list(map(list,r_ls))
    r_ls = reverse_coordinates(input=r_ls)
    return r_ls

def return_ids(ids,ls):
    r_ls = [ids[each] for each in ls]
    return r_ls

def call_api(coordinates):

  body = {"coordinates": coordinates}

  headers = {
      'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
      'Authorization': '5b3ce3597851110001cf624822b79fbe37204f6a83490d4a1566d9a5',
      'Content-Type': 'application/json; charset=utf-8'
  }
  call = requests.post('https://api.openrouteservice.org/v2/directions/driving-car', json=body, headers=headers)

  # print(call.status_code, call.reason)
  if call.status_code == 200:
    response = json.loads(call.text)
    geometry = response['routes'][0]['geometry']
    distance = round(response['routes'][0]['summary']['distance']/1000,1)
    duration = round(response['routes'][0]['summary']['duration']/60,1)
  else:
    print("API call failed for this list of coordinates coordinates: ", coordinates)
    print(call.text)
    geometry = None
    distance = 1
    duration = 1

  return geometry, distance, duration


op = output
error = 0
suc = 0
output_1 = {}
for k, v in op.items():
    output_1[k] = {}
    #print(k, " and", v)
    #print(v)
    for k2, v2 in v.items():
        if k2 == 'route_plan':
          #time.sleep(2)
          #print(k2, v2)
          #print(return_coords(coordinates, op[k]['route_plan']))
          if len(return_coords(coordinates ,op[k]['route_plan'])) > 1:
              api_response = call_api(return_coords(coordinates, op[k]['route_plan']))
              output_1[k]['route_geometry'] = api_response[0]
              output_1[k]['route_distance'] = api_response[1]
              output_1[k]['route_duration'] = api_response[2]
          else:
              output_1[k]['route_geometry'] = 0
              output_1[k]['route_distance'] = 0
              output_1[k]['route_duration'] = 0
        output_1[k]['route_plan'] = op[k]['route_plan']
        output_1[k]['cumulative_route_load'] = op[k]['cumulative_route_load']

output_1

API call failed for this list of coordinates coordinates:  [[55.06185223067843, 24.931690466392208], ['55.16941096', '24.97328883'], ['55.18360903', '24.98594083'], ['55.8023118', '25.2196159'], ['55.85933605', '25.28101319'], ['55.87840707', '25.28138983'], ['55.87972065', '25.28470953'], ['56.33393486', '25.16244746'], ['56.33692932', '25.16572974'], ['56.33979114', '25.16425763'], ['56.33976194', '25.16657223'], ['56.34810491', '25.23825772'], ['56.3601253', '25.271208'], ['56.36021631', '25.27326995'], ['56.36214435', '25.27359185'], ['56.36253168', '25.28865974'], ['56.36497572', '25.29275516'], ['56.3640989', '25.2927652'], ['56.36264719', '25.29495949'], ['56.34961639', '25.34440223'], ['56.35427397', '25.34548413'], ['56.35522958', '25.51571477'], ['56.2616838', '25.5797136'], ['55.85335866', '25.35263409'], ['55.85301838', '25.35209073'], ['55.85257973', '25.34117057'], ['55.4192414002395', '25.1610695662702'], ['55.4077794961631', '25.1568435737863'], ['55.380675000000004', '

{'0': {'route_geometry': 0,
  'route_distance': 0,
  'route_duration': 0,
  'route_plan': [0],
  'cumulative_route_load': [0]},
 '1': {'route_geometry': 0,
  'route_distance': 0,
  'route_duration': 0,
  'route_plan': [0],
  'cumulative_route_load': [0]},
 '2': {'route_geometry': 0,
  'route_distance': 0,
  'route_duration': 0,
  'route_plan': [0],
  'cumulative_route_load': [0]},
 '3': {'route_geometry': 0,
  'route_distance': 0,
  'route_duration': 0,
  'route_plan': [0],
  'cumulative_route_load': [0]},
 '4': {'route_geometry': 0,
  'route_distance': 0,
  'route_duration': 0,
  'route_plan': [0],
  'cumulative_route_load': [0]},
 '5': {'route_geometry': 0,
  'route_distance': 0,
  'route_duration': 0,
  'route_plan': [0],
  'cumulative_route_load': [0]},
 '6': {'route_geometry': 0,
  'route_distance': 0,
  'route_duration': 0,
  'route_plan': [0],
  'cumulative_route_load': [0]},
 '7': {'route_geometry': 0,
  'route_distance': 0,
  'route_duration': 0,
  'route_plan': [0],
  'cumula